# Solution 07: Regularization

In [7]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

from IPython.display import display

pd.options.display.max_columns = 50

In [8]:
df = pd.read_csv("../data/immo_data.csv")
desc = pd.read_csv("../data/immo_data_column_description.csv")

In [9]:
def drop_columns(df):
    """ Remove (supposedly) unimportant columns """
    return df.drop(
        [
            "scoutId",
            "houseNumber",
            "geo_bln",
            "geo_krs",
            "geo_plz",
            "date",
            "street",
            "streetPlain",
            "description",
            "facilities",
            "regio3",
            "firingTypes",
            "telekomHybridUploadSpeed",
            "totalRent",
            "baseRentRange",
        ],
        axis=1,
    )


def remove_outliers(df, lower_limit=0.005, upper_limit=0.995):
    """ Removing the (lower and upper) outliers """
    dfc = df.copy()
    columns_with_outliers = [
        "serviceCharge",
        "yearConstructed",
        "noParkSpaces",
        "baseRent",
        "livingSpace",
        "noRooms",
        "floor",
        "numberOfFloors",
        "heatingCosts",
        "lastRefurbish",
    ]
    
    # For each column we keep: Data that are < (99.5% quantile) and > (0.5% quantile) OR that are NaN (we will deal with this later). 
    upper_limits = df[columns_with_outliers].quantile(upper_limit)
    lower_limits = df[columns_with_outliers].quantile(lower_limit)
    
    for colname in columns_with_outliers:
        col = dfc[colname]
        dfc = dfc[
            ((col <= upper_limits[colname]) & (col >= lower_limits[colname]))
            | col.isna()
        ]
    return dfc


def remove_rows_with_NaN_target(df):
    """ Removing the records without a label """
    return df[df["baseRent"].isna() == False]


def impute_NaNs(df):
    """ Replacing NaNs with mean or most frequent """
    dfc = df.copy()
    categorical_columns = dfc.select_dtypes(exclude=np.number).columns
    imp_freq = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
    dfc[categorical_columns] = imp_freq.fit_transform(dfc[categorical_columns])

    numeric_columns = dfc.select_dtypes(include=np.number).columns
    imp_mean = SimpleImputer(missing_values=np.nan, strategy="mean")
    dfc[numeric_columns] = imp_mean.fit_transform(dfc[numeric_columns])
    return dfc


def print_evaluation(pipeline_or_model, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names, show_coeff=True):
    """ Output of R2 value, MSE and MAE for training and test set """
    r2_train = r2_score(y_train, y_train_pred)
    mse_train = math.sqrt(mean_squared_error(y_train, y_train_pred))
    mae_train = mean_absolute_error(y_train, y_train_pred)

    r2_test = r2_score(y_test, y_test_pred)
    mse_test = math.sqrt(mean_squared_error(y_test, y_test_pred))
    mae_test = mean_absolute_error(y_test, y_test_pred)
    
    print(
        f"{pipeline_or_model} Evaluation:\n"
        f"{'':6} {'R²':>10} | {'MSE':>14} | {'MAE':>10} | {'rows':>8} | {'columns':>8}\n"
        f"{'Train':6} {r2_train:10.5f} | {mse_train:14.2f} | {mae_train:10.2f} | {X_train.shape[0]:8} | {X_train.shape[1]:8}\n"
        f"{'Test':6} {r2_test:10.5f} | {mse_test:14.2f} | {mae_test:10.2f} | {X_test.shape[0]:8} | {X_test.shape[1]:8}\n"
    )
    
    # Output of the first 10 coefficients, scaled in descending order by absolute value
    coefficients_lr = pd.DataFrame({"Feature Name": feature_names, "Coefficient": pipeline_or_model.coef_})
    if show_coeff:
        display(coefficients_lr.sort_values("Coefficient", key=abs, ascending=False).head(10))
    
    # How many coefficients are non-zero?
    nonzero_coeff = sum(abs(coefficients_lr["Coefficient"])>1e-12)
    print(f"Number of nonzero coefficients: {nonzero_coeff}/{X_train.shape[1]}")

## Task 1 
Observation: with increasing $\alpha$ the error on the training data increases. In this data set, the problem of immens error on the test data is already eliminated with a very small value for $\alpha$. The larger we make $\alpha$, the larger errors on training AND test data will eventually become. With the $\ell_1$-regularization (LASSO), however, more and more coefficients are equal to zero, i.e. a model with few features (=easier to understand for humans) already has a high predictive power.

In [10]:
# Data pre-processing
df_reduced = drop_columns(df.sample(10000, random_state=42))
df_reduced = remove_outliers(df_reduced)
df_reduced = remove_rows_with_NaN_target(df_reduced)
df_reduced = impute_NaNs(df_reduced)
df_reduced = pd.get_dummies(df_reduced)
y = df_reduced.pop("baseRent")

# Training-Test-Split
X_train, X_test, y_train, y_test = train_test_split(df_reduced, y, test_size=0.2, random_state=0)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
# equivalent: .fit_transform calls .fit first, then .transform
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Training for difference valus of alpha
N = X_train.shape[0]
alphas = [0, 1e-3, 1e-2, 1e-1, 1, 10, 100]
models = [linear_model.Ridge, linear_model.Lasso]
factors = [np.sqrt(N), 1]
for model, factor in zip(models, factors):
    for alpha in alphas:
        model_lr = model(alpha=factor*alpha)
        model_lr.fit(X_train, y_train)
        y_train_pred = model_lr.predict(X_train)
        y_test_pred = model_lr.predict(X_test)

        # Evaluation
        print(f"{model}, alpha = {factor*alpha}")
        print_evaluation(model_lr, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names=df_reduced.columns, show_coeff=False)
        print("--------------------------------------------")

<class 'sklearn.linear_model._ridge.Ridge'>, alpha = 0.0
Ridge(alpha=0.0) Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.82795 |         167.97 |     114.18 |     7742 |      511
Test   -1024275060639381195849728.00000 | 407179081533658.12 | 26006931164852.94 |     1936 |      511

Number of nonzero coefficients: 511/511
--------------------------------------------
<class 'sklearn.linear_model._ridge.Ridge'>, alpha = 0.08798863562983575
Ridge(alpha=0.08798863562983575) Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.84461 |         159.62 |     103.83 |     7742 |      511
Test      0.82224 |         169.62 |     114.30 |     1936 |      511

Number of nonzero coefficients: 502/511
--------------------------------------------
<class 'sklearn.linear_model._ridge.Ridge'>, alpha = 0.8798863562983574
Ridge(alpha=0.8798863562983574) Evaluation:
               R² |            MSE |        MAE |     r

/var/folders/5b/zrc1vq6j1gg62hcmsxkpdr4m0000gn/T/ipykernel_40818/53933199.py:28: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  model_lr.fit(X_train, y_train)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.863e+07, tolerance: 1.269e+05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordi

<class 'sklearn.linear_model._coordinate_descent.Lasso'>, alpha = 0
Lasso(alpha=0) Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.84461 |         159.62 |     103.83 |     7742 |      511
Test      0.82104 |         170.20 |     114.65 |     1936 |      511

Number of nonzero coefficients: 498/511
--------------------------------------------


/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.890e+07, tolerance: 1.269e+05
  model = cd_fast.enet_coordinate_descent(


<class 'sklearn.linear_model._coordinate_descent.Lasso'>, alpha = 0.001
Lasso(alpha=0.001) Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.84461 |         159.62 |     103.82 |     7742 |      511
Test      0.82136 |         170.05 |     114.56 |     1936 |      511

Number of nonzero coefficients: 497/511
--------------------------------------------


/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.683e+06, tolerance: 1.269e+05
  model = cd_fast.enet_coordinate_descent(


<class 'sklearn.linear_model._coordinate_descent.Lasso'>, alpha = 0.01
Lasso(alpha=0.01) Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.84461 |         159.62 |     103.81 |     7742 |      511
Test      0.82246 |         169.52 |     114.21 |     1936 |      511

Number of nonzero coefficients: 482/511
--------------------------------------------
<class 'sklearn.linear_model._coordinate_descent.Lasso'>, alpha = 0.1
Lasso(alpha=0.1) Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.84457 |         159.64 |     103.72 |     7742 |      511
Test      0.82291 |         169.31 |     113.96 |     1936 |      511

Number of nonzero coefficients: 462/511
--------------------------------------------
<class 'sklearn.linear_model._coordinate_descent.Lasso'>, alpha = 1
Lasso(alpha=1) Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.84186 |         161.03 |     10

## Task 2

In [11]:
from sklearn.model_selection import KFold

def cross_validation(alphas, X_train, y_train):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mean_scores = []
    stddev_scores = []
    
    # Loop over all the alphas we want to try out
    for alpha in alphas:
        scores = []
        # Loop over 5 different training/test splits
        for train_index, test_index in kf.split(X_train):
            X_trainK, X_testK = X_train[train_index], X_train[test_index]
            y_trainK, y_testK = y_train[train_index], y_train[test_index]
            m_ridgeK = linear_model.Ridge(alpha=alpha)
            m_ridgeK.fit(X_trainK, y_trainK)
            score = m_ridgeK.score(X_testK, y_testK)
            scores.append(score)
        # Saving mean and standard deviation over the 5 runs for each alpha
        mean_scores.append(np.mean(np.array(scores)))
        stddev_scores.append(np.std(np.array(scores)))
    return mean_scores, stddev_scores

# Data pre-processing
df_reduced = drop_columns(df.sample(10000, random_state=42))
df_reduced = remove_outliers(df_reduced)
df_reduced = remove_rows_with_NaN_target(df_reduced)
df_reduced = impute_NaNs(df_reduced)
df_reduced = pd.get_dummies(df_reduced)
y = df_reduced.pop("baseRent").values

# Training-Test-Split
X_train, X_test, y_train, y_test = train_test_split(df_reduced, y, test_size=0.2, random_state=0)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Cross validation (step 2 algorithm)
alphas = [1e-3, 1e-2, 1e-1, 1, 10, 100]
mean_scores, stddev_scores = cross_validation(alphas, X_train, y_train)

# For the best alpha we train again on the full training dataset
best_alpha_index = mean_scores.index(max(mean_scores))
best_alpha = alphas[best_alpha_index]
m_ridge = linear_model.Ridge(alpha=best_alpha)
m_ridge.fit(X_train, y_train)
y_train_pred = m_ridge.predict(X_train)
y_test_pred = m_ridge.predict(X_test)
print(f"Best alpha={best_alpha} (mean score={mean_scores[best_alpha_index]}, sigma={stddev_scores[best_alpha_index]}")

print_evaluation(m_ridge, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names=df_reduced.columns, show_coeff=False)

Best alpha=10 (mean score=0.8186885840835261, sigma=0.013342189818551273
Ridge(alpha=10) Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.84459 |         159.63 |     103.79 |     7742 |      511
Test      0.82232 |         169.59 |     114.24 |     1936 |      511

Number of nonzero coefficients: 502/511


## Task 3- Nested Cross-Validation
Here an outer loop around the cross validation is implemented. The purpose is that different test sets are calculated. The average over the errors on the different test sets is now a more reliable estimate for the error on unknown data. One disadvantage is that you end up with multiple models. For the deployment you have to choose one.

In [12]:
# Data pre-processing
df_reduced = drop_columns(df.sample(10000, random_state=42))
df_reduced = remove_outliers(df_reduced)
df_reduced = remove_rows_with_NaN_target(df_reduced)
df_reduced = impute_NaNs(df_reduced)
df_reduced = pd.get_dummies(df_reduced)
y = df_reduced.pop("baseRent").values

X = df_reduced.values.astype(float)

# Outer loop cross validation: Various test set

kf_outer = KFold(n_splits=3, shuffle=True, random_state=42)
test_scores = []
# Loop over 3 different training/test splits
for train_index, test_index in kf_outer.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Inner cross validation to determine the best alpha (step 2 algorithm).
    alphas = [1e-3, 1e-2, 1e-1, 1, 10, 100]
    mean_scores, stddev_scores = cross_validation(alphas, X_train, y_train)

    # For the best alpha we train again on the full training dataset
    best_alpha_index = mean_scores.index(max(mean_scores))
    best_alpha = alphas[best_alpha_index]
    m_ridge = linear_model.Ridge(alpha=best_alpha)
    m_ridge.fit(X_train, y_train)
    y_train_pred = m_ridge.predict(X_train)
    y_test_pred = m_ridge.predict(X_test)
    test_scores.append(m_ridge.score(X_test, y_test))

    print(f"Best alpha={best_alpha} (mean score={mean_scores[best_alpha_index]}, sigma={stddev_scores[best_alpha_index]}")
    print_evaluation(m_ridge, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names=df_reduced.columns, show_coeff=False)
    print(f"Mean of R² on test sets: {np.mean(np.array(test_scores))}")

Best alpha=10 (mean score=0.813919081347251, sigma=0.010418081871166078
Ridge(alpha=10) Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.84341 |         161.87 |     105.05 |     6452 |      511
Test      0.82408 |         165.60 |     112.50 |     3226 |      511

Number of nonzero coefficients: 498/511
Mean of R² on test sets: 0.8240823199994203
Best alpha=10 (mean score=0.8233035341808088, sigma=0.010934817423729495
Ridge(alpha=10) Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.85149 |         154.99 |     102.29 |     6452 |      511
Test      0.81204 |         177.21 |     114.17 |     3226 |      511

Number of nonzero coefficients: 504/511
Mean of R² on test sets: 0.8180590734560631
Best alpha=10 (mean score=0.8127459941121421, sigma=0.012346249054763652
Ridge(alpha=10) Evaluation:
               R² |            MSE |        MAE |     rows |  columns
Train     0.84154 |         160.00 |  